# Screencast Code

The follow code is the same used in the "Numeric Features" screencast. Run each code cell to see how 

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, VectorAssembler, Normalizer, StandardScaler
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, StringType

import re

In [2]:
# create a SparkSession: note this step was left out of the screencast
spark = SparkSession.builder \
    .master("local") \
    .appName("Word Count") \
    .getOrCreate()

# Read in the Data Set

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)

In [5]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php')

In [7]:
df.printSchema()

root
 |-- Body: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- oneTag: string (nullable = true)



In [8]:
df.show(1)

+--------------------+---+--------------------+--------------------+------+
|                Body| Id|                Tags|               Title|oneTag|
+--------------------+---+--------------------+--------------------+------+
|<p>I'd like to ch...|  1|php image-process...|How to check if a...|   php|
+--------------------+---+--------------------+--------------------+------+
only showing top 1 row



# Tokenization

Tokenization splits strings into separate words. Spark has a [Tokenizer](https://spark.apache.org/docs/latest/ml-features.html#tokenizer) class as well as RegexTokenizer, which allows for more control over the tokenization process.

In [9]:
# split the body text into separate words
regexTokenizer = RegexTokenizer(inputCol='Body', outputCol='words', pattern='\\W')
df = regexTokenizer.transform(df)

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which

In [10]:
df.show(1)

+--------------------+---+--------------------+--------------------+------+--------------------+
|                Body| Id|                Tags|               Title|oneTag|               words|
+--------------------+---+--------------------+--------------------+------+--------------------+
|<p>I'd like to ch...|  1|php image-process...|How to check if a...|   php|[p, i, d, like, t...|
+--------------------+---+--------------------+--------------------+------+--------------------+
only showing top 1 row



In [19]:
# count the number of words in each body tag

body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("BodyLength", body_length(df.words))

In [20]:
# count the number of paragraphs and links in each body tag

number_of_paragraphs = udf(lambda x: len(re.findall("</p>", x)), IntegerType())
number_of_links = udf(lambda x: len(re.findall("</a>", x)), IntegerType())

In [21]:
df = df.withColumn("NumParagraphs", number_of_paragraphs(df.Body))
df = df.withColumn("NumLinks", number_of_links(df.Body))

In [23]:
df.show(2)

+--------------------+---+--------------------+--------------------+-------+--------------------+---------+----------+-------------+--------+
|                Body| Id|                Tags|               Title| oneTag|               words|BodyLenth|BodyLength|NumParagraphs|NumLinks|
+--------------------+---+--------------------+--------------------+-------+--------------------+---------+----------+-------------+--------+
|<p>I'd like to ch...|  1|php image-process...|How to check if a...|    php|[p, i, d, like, t...|       83|        83|            2|       0|
|<p>In my favorite...|  2|             firefox|How can I prevent...|firefox|[p, in, my, favor...|       71|        71|            2|       0|
+--------------------+---+--------------------+--------------------+-------+--------------------+---------+----------+-------------+--------+
only showing top 2 rows



# VectorAssembler

Combine the body length, number of paragraphs, and number of links columns into a vector

In [24]:
assembler = VectorAssembler(inputCols=["BodyLength", "NumParagraphs", "NumLinks"], outputCol="NumFeatures")
df = assembler.transform(df)

In [25]:
df.show(1)

+--------------------+---+--------------------+--------------------+------+--------------------+---------+----------+-------------+--------+--------------+
|                Body| Id|                Tags|               Title|oneTag|               words|BodyLenth|BodyLength|NumParagraphs|NumLinks|   NumFeatures|
+--------------------+---+--------------------+--------------------+------+--------------------+---------+----------+-------------+--------+--------------+
|<p>I'd like to ch...|  1|php image-process...|How to check if a...|   php|[p, i, d, like, t...|       83|        83|            2|       0|[83.0,2.0,0.0]|
+--------------------+---+--------------------+--------------------+------+--------------------+---------+----------+-------------+--------+--------------+
only showing top 1 row



# Normalize the Vectors

In [26]:
scaler = Normalizer(inputCol="NumFeatures", outputCol="ScaledNumFeatures")
df = scaler.transform(df)

In [27]:
df.show(1)

+--------------------+---+--------------------+--------------------+------+--------------------+---------+----------+-------------+--------+--------------+--------------------+
|                Body| Id|                Tags|               Title|oneTag|               words|BodyLenth|BodyLength|NumParagraphs|NumLinks|   NumFeatures|   ScaledNumFeatures|
+--------------------+---+--------------------+--------------------+------+--------------------+---------+----------+-------------+--------+--------------+--------------------+
|<p>I'd like to ch...|  1|php image-process...|How to check if a...|   php|[p, i, d, like, t...|       83|        83|            2|       0|[83.0,2.0,0.0]|[0.99970980846748...|
+--------------------+---+--------------------+--------------------+------+--------------------+---------+----------+-------------+--------+--------------+--------------------+
only showing top 1 row



# Scale the Vectors

In [28]:
scaler2 = StandardScaler(inputCol="NumFeatures", outputCol="ScaledNumFeatures2", withStd=True)
scalerModel = scaler2.fit(df)
df = scalerModel.transform(df)

In [29]:
df.show(1)

+--------------------+---+--------------------+--------------------+------+--------------------+---------+----------+-------------+--------+--------------+--------------------+--------------------+
|                Body| Id|                Tags|               Title|oneTag|               words|BodyLenth|BodyLength|NumParagraphs|NumLinks|   NumFeatures|   ScaledNumFeatures|  ScaledNumFeatures2|
+--------------------+---+--------------------+--------------------+------+--------------------+---------+----------+-------------+--------+--------------+--------------------+--------------------+
|<p>I'd like to ch...|  1|php image-process...|How to check if a...|   php|[p, i, d, like, t...|       83|        83|            2|       0|[83.0,2.0,0.0]|[0.99970980846748...|[0.43245858153237...|
+--------------------+---+--------------------+--------------------+------+--------------------+---------+----------+-------------+--------+--------------+--------------------+--------------------+
only showi